In [2]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import urllib2
from multiprocessing import Pool as ThreadPool
import math
import cPickle
import os

In [6]:
priceArr = []
patCollect = []
endingInd = []
patLen = 10
futureE = 10
quoteCollect = {}
bestArr = []

def percentChange(startPoint,currentPoint):
    try:
        x = ((float(currentPoint)-startPoint)/abs(startPoint))
        return x
    except:
        print(currentPoint, startPoint)
        return 0.00000000001

def loadPats():
    global priceArr

    priceArr = []
    with open(r"one.pickle", "rb") as input_file:
        e = cPickle.load(input_file)
        priceArr = e


def collectPats():
    global patCollect
    global endingInd
    patCollect = []
    endingInd = []
    for each in priceArr:
        tempCollect = []
        tempEnd = []
        sIndex = 0
        length = (len(each) - patLen - futureE)

        while sIndex < length:
            inc = 1
            tempPat = []
            while inc <= patLen:
                temp = percentChange(each[sIndex + inc -1], each[sIndex + inc])
                inc += 1
                tempPat.append(temp)
            tempCollect.append(tempPat)
            tempEnd.append(sIndex+patLen)
            sIndex += (patLen)
        patCollect.append(tempCollect)
        endingInd.append(tempEnd)

loadPats()
collectPats()

In [7]:
def yahooLoad(val):
    try:
        tempArr = []
        string = ""
        string = 'http://ichart.finance.yahoo.com/table.csv?s={0}'.format(val)
        count = 0
        for line in urllib2.urlopen(string):
            if count == 0:
                count = 1
                continue
            close = line.split(',')[6]
            close = float(str(close))
            tempArr.append(close)
        tempArr = tempArr[::-1]
        #quoteCollect[val] = tempArr
        return {val: tempArr[-100:]}
    except:
        {'error': 'error'}
        pass


def currentPat(curArr):
    global patLen
    curPat = []
    sliceLen = patLen
    curr = curArr[-(sliceLen+1):]
    if curr[-3] == curr[-1]:
        return ""
    i = 0
    while i < patLen:
        temp = percentChange(curr[i], curr[i + 1])
        curPat.append(temp)
        i += 1
    return curPat


def sortPats(array):
    finalArr = []
    def getKey(tup):
        return tup[1]
    finalArr = sorted(array, key=getKey)
    return finalArr


def matchPats(patCollect,endingInd,curPat):
    matchedPat = []
    matchedEndInd = []
    bestMatches = []
    for colInd,col in enumerate(patCollect):
        for rowInd,row in enumerate(col):
            mseCollect = []
            for ind,item in enumerate(row):
                mseTemp = abs(curPat[ind] - item)
                mseCollect.append(mseTemp)
            #mseAvg = np.average(weighting)
            mseAvg = np.average(mseCollect)
            bestMatches.append(([row,colInd,endingInd[colInd][rowInd]],mseAvg))
    test = sortPats(bestMatches)
    for item in test[:10]:
        matchedPat.append(item[0][0])
        matchedEndInd.append((item[0][1],item[0][2]))
    return matchedPat,matchedEndInd

In [8]:
def plotting(matchedPat,matchedEndInd):
    global futureE
    futureAverages = []
    stDev = []
    futureLine = []
    futurePercent = []
    if len(matchedPat) > 1:
        for i in matchedPat:
            tempPercent = []
            refInd = matchedPat.index(i)
            endInd = matchedEndInd[refInd]
            col = endInd[0]
            outcomesZ = priceArr[col][endInd[1]:(endInd[1]+patLen)]
            futureLine.append(outcomesZ)
            for x in outcomesZ:
                change = percentChange(priceArr[col][endInd[1]],x)
                tempPercent.append(change)
            futurePercent.append(tempPercent)

        futureAverages = []

        for arrItemNum in range(len(futurePercent[0])):
            tempOutcomes = []
            for arr in futurePercent:
                tempOutcomes.append(arr[arrItemNum])
            futureAverages.append(np.mean(tempOutcomes))
            stDev.append(np.std(tempOutcomes))

        return futureAverages,stDev

In [9]:
def sortBest(array):
    finalArr = []
    def getKey(val):
        return val['future'][-1]
    finalArr = sorted(array, key=getKey, reverse=True)
    return finalArr

bestCollect = []

def bestGo(val,curArr):
    global bestCollect
    try:
        #curArr = yahooLoad(val)
        curPat = currentPat(curArr)
        matchedPat,matchedEndInd = matchPats(patCollect,endingInd,curPat)
        futureAverages,stDev = plotting(matchedPat,matchedEndInd)
        totalDict = {'symbol': val,'matches': matchedPat,'current': curPat,'future': futureAverages, 'stDev': stDev, 'sharpe': futureAverages[-1]/stDev[-1]}
        bestCollect.append(totalDict)
    except:
        bestCollect.append({'error':'error'})
        pass

In [10]:
arr = ['SPY','TLT','AAPL','GOOGL','AMZN','NFLX','BAC','JPM','MCD','TSLA','MCD','NKE','MSFT',
  'XLE','XLF','QQQ','FB','VZ','GE','BA']#,'HD','DIS','JNJ','GS','PCLN','MS','TJX','M','SBUX','XOM','V','MA',
  #'XLU','IBM','INTC','XLV','XLI','IYR','XLY','EEM','FXI','GLD','SLV','GDX','FXE','UUP','HYG','SMH']

In [18]:
pool = ThreadPool(4)
quoteCollect = pool.map(yahooLoad, arr)
pool.close()
pool.join()

bestCollect = []
for i in quoteCollect:
    try:
        for key,value in i.iteritems():
            if value != 'error':
                bestGo(key,value)
    except:
        continue

finalBest = []
for val in bestCollect:
    if 'error' not in val:
        if val['sharpe'] > 0.85:
            finalBest.append(val)
bestArr = sortBest(finalBest)


In [19]:
weights = {}
count = 0
for line in urllib2.urlopen('file:///Users/sjcasella/Desktop/TradeViz_P/spxw.csv'):
    nLine = (line.replace('"','')).replace('%','')
    if count == 0:
        count = 1
        continue
    ticker = str(nLine.split(',')[0])
    try:
        weight = float(nLine.split(',')[2])/100
    except:
        continue
    weights[ticker] = weight

In [20]:
tempArr = {}
for i in bestCollect:
    if 'error' in i:
        continue
    tempVals = {}
    val = i['future'] 
    tempArr[i['symbol']] = val

totalW = 0.0
for key,val in tempArr.iteritems():
    if key in weights:
        totalW = totalW + weights[key]

spxLine = []
for key,val in tempArr.iteritems():
    if key in weights:
        weight = weights[key]/totalW
        spxLine.append([i * weight for i in val])

In [21]:
spxLen = len(spxLine[0])
spxFinal = []
for ind in range(spxLen):
    totalEach = 0.0
    for subArr in spxLine:
        totalEach = totalEach + subArr[ind]
    spxFinal.append(totalEach)


In [22]:
bestArr.append({'symbol':'SPXLINE','future':spxFinal})

os.remove("best.pickle")
with open(r"best.pickle", "wb") as output_file:
    cPickle.dump(bestArr, output_file)

In [23]:
print(bestArr)

[{'symbol': 'SPXLINE', 'future': [0.0, 0.0044397241716545438, 0.0014197823332921461, 0.0028117473386316911, 0.0042154607903476405, 0.0028180968975619332, 0.0037881659868702253, 0.0027846909443363576, 0.00087314279726220952, 0.0033808267158673263]}]


In [5]:
import json
content = urllib2.urlopen('http://finance.google.com/finance/info?client=ig&q=AAPL,GOOG')
dat = content
print(json.dumps(dat))

TypeError: <addinfourl at 4417071512 whose fp = <socket._fileobject object at 0x107465950>> is not JSON serializable

In [10]:

u = urllib2.urlopen('http://finance.google.com/finance/info?client=ig&q=AAPL,GOOG')
content = u.read()
obj = json.loads(content[3:])
print(json.dumps(obj))
        


// [
{
"id": "22144"
,"t" : "AAPL"
,"e" : "NASDAQ"
,"l" : "97.42"
,"l_fix" : "97.42"
,"l_cur" : "97.42"
,"s": "1"
,"ltt":"4:00PM EDT"
,"lt" : "Jul 12, 4:00PM EDT"
,"lt_dts" : "2016-07-12T16:00:01Z"
,"c" : "+0.44"
,"c_fix" : "0.44"
,"cp" : "0.45"
,"cp_fix" : "0.45"
,"ccol" : "chg"
,"pcls_fix" : "96.98"
,"el": "97.88"
,"el_fix": "97.88"
,"el_cur": "97.88"
,"elt" : "Jul 13, 6:41AM EDT"
,"ec" : "+0.46"
,"ec_fix" : "0.46"
,"ecp" : "0.47"
,"ecp_fix" : "0.47"
,"eccol" : "chg"
,"div" : "0.57"
,"yld" : "2.34"
}
,{
"id": "304466804484872"
,"t" : "GOOG"
,"e" : "NASDAQ"
,"l" : "720.64"
,"l_fix" : "720.64"
,"l_cur" : "720.64"
,"s": "1"
,"ltt":"4:00PM EDT"
,"lt" : "Jul 12, 4:00PM EDT"
,"lt_dts" : "2016-07-12T16:00:01Z"
,"c" : "+5.55"
,"c_fix" : "5.55"
,"cp" : "0.78"
,"cp_fix" : "0.78"
,"ccol" : "chg"
,"pcls_fix" : "715.09"
,"el": "722.55"
,"el_fix": "722.55"
,"el_cur": "722.55"
,"elt" : "Jul 13, 5:04AM EDT"
,"ec" : "+1.91"
,"ec_fix" : "1.91"
,"ecp" : "0.27"
,"ecp_fix" : "0.27"
,"eccol" : "chg"
,"di

ValueError: No JSON object could be decoded